In [2]:
#pip install pandahouse  # если библиотека ещё не установлена
import pandahouse as ph
# Здесь презентация: CR ARPU students AB test
import time # для иззмерения времени запроса

In [3]:
connection_default = {'host': 'https://clickhouse.lab.karpov.courses',
                      'database':'default',
                      'user':'student', 
                      'password':'dpo_python_2020'
                     }

In [28]:
# создаем запрос, db автоматически заменится на значение из database
# нет особого смысла в AND toMonth(timest) = 10, но в условии прописано про "в этом месяце"
# и вообще там тогда должно быть AND toMonth(timest) = toMonth(now()), но на апрель 2023 года в БД записей нет - выводит 0
# сделал 2 запроса
start_time = time.time() # начало исполнения запроса

q = '''
--SELECT MAX(timest) FROM default.peas -- 2021-10-31 17:29:52.000 самая поздняя дата в таблице
--SELECT MIN(timest) FROM default.peas -- 2021-10-30 13:00:00.000 самая ранняя дата в таблице

SELECT
    COUNT(diligence) AS count_of_eager_students
FROM(
SELECT 
    SUM(correct) AS diligence
FROM default.peas
GROUP BY st_id
HAVING diligence >= 20 AND toMonth(timest) = 10) AS t
    '''

# отправляем запрос и записываем результат в пандасовский датафрейм
diligence = ph.read_clickhouse(query=q, connection=connection_default)

end_time = time.time() # конец исполнения запроса
elapsed_time = end_time - start_time #  время запроса

print("Время выполнения запроса: {:.2f} секунд".format(elapsed_time))

diligence

Время выполнения запроса: 0.07 секунд


,count_of_eager_students
0,136


In [29]:
# создаем запрос, db автоматически заменится на значение из database
# нет особого смысла в AND toMonth(timest) = 10, но в условии прописано про "в этом месяце"
# и вообще там тогда должно быть AND toMonth(timest) = toMonth(now()), но на апрель 2023 года в БД записей нет - выводит 0
start_time = time.time() # начало исполнения запроса
q = '''
--SELECT MAX(timest) FROM default.peas -- 2021-10-31 17:29:52.000 самая поздняя дата в таблице
--SELECT MIN(timest) FROM default.peas -- 2021-10-30 13:00:00.000 самая ранняя дата в таблице

SELECT COUNT(st_id) AS count_of_eager_students
FROM (
    SELECT st_id
    FROM (
        SELECT st_id, SUM(correct) AS diligence
        FROM default.peas
        WHERE toMonth(timest) = 10
        GROUP BY st_id
    ) AS t
    WHERE diligence >= 20
) AS t2
    '''

# отправляем запрос и записываем результат в пандасовский датафрейм
diligence = ph.read_clickhouse(query=q, connection=connection_default)

end_time = time.time() # конец исполнения запроса
elapsed_time = end_time - start_time #  время запроса

print("Время выполнения запроса: {:.2f} секунд".format(elapsed_time))

diligence

Время выполнения запроса: 0.08 секунд


,count_of_eager_students
0,136


In [27]:
# создаем запрос, db автоматически заменится на значение из database
q = '''
WITH sq1 AS (
SELECT 
    ROUND(SUM(money) / COUNT(DISTINCT default.peas.st_id), 2) AS ARPU_control
FROM default.final_project_check AS fpc
FULL JOIN default.peas AS p USING(st_id)
FULL JOIN default.studs ON default.peas.st_id = default.studs.st_id
WHERE test_grp = 'control'), 


sq2 AS (
SELECT 
    ROUND(SUM(money) / COUNT(DISTINCT default.peas.st_id), 2) AS ARPU_pilot
FROM default.final_project_check AS fpc
FULL JOIN default.peas AS p USING(st_id)
FULL JOIN default.studs ON default.peas.st_id = default.studs.st_id
WHERE test_grp = 'pilot'),

sq3 AS (SELECT 
    ROUND(SUM(money) / COUNT(DISTINCT default.peas.st_id), 2) AS ARPU_active_control
FROM default.final_project_check AS fpc
FULL JOIN default.peas AS p USING(st_id)
FULL JOIN default.studs ON default.peas.st_id = default.studs.st_id
WHERE default.peas.st_id IN (SELECT st_id FROM default.peas GROUP BY st_id HAVING SUM(correct) > 10) AND test_grp = 'control'),

sq4 AS (
SELECT 
    ROUND(SUM(money) / COUNT(DISTINCT default.peas.st_id), 2) AS ARPU_active_pilot
FROM default.final_project_check AS fpc
FULL JOIN default.peas AS p USING(st_id)
FULL JOIN default.studs ON default.peas.st_id = default.studs.st_id
WHERE default.peas.st_id IN (SELECT st_id FROM default.peas GROUP BY st_id HAVING SUM(correct) > 10) AND test_grp = 'pilot'),

sq5 AS (SELECT -- 9.64%  Конверсия из юзера в покупателя - control group
        ROUND(fpc.payd_users / p.all_users * 100, 2) AS CR_control
FROM
(SELECT 
    COUNT(DISTINCT default.peas.st_id) AS all_users 
FROM default.peas
FULL JOIN default.final_project_check USING(st_id)
FULL JOIN default.studs ON default.peas.st_id = default.studs.st_id
WHERE test_grp = 'control'
) AS p
CROSS JOIN 
(SELECT 
    COUNT(DISTINCT st_id) AS payd_users
FROM default.final_project_check
FULL JOIN default.studs ON default.final_project_check.st_id = default.studs.st_id
WHERE test_grp = 'control'
) AS fpc),

sq6 AS (SELECT -- 22.60%  Конверсия из юзера в покупателя - test group
        ROUND(fpc.payd_users / p.all_users * 100, 2) AS CR_pilot
FROM
(SELECT 
    COUNT(DISTINCT default.peas.st_id) AS all_users 
FROM default.peas
FULL JOIN default.final_project_check USING(st_id)
FULL JOIN default.studs ON default.peas.st_id = default.studs.st_id
WHERE test_grp = 'pilot'
) AS p
CROSS JOIN 
(SELECT 
    COUNT(DISTINCT st_id) AS payd_users
FROM default.final_project_check
FULL JOIN default.studs ON default.final_project_check.st_id = default.studs.st_id
WHERE test_grp = 'pilot'
) AS fpc),

sq7 AS (SELECT -- 12.60%  Конверсия из активного юзера в покупателя - control group
        ROUND(fpc.payd_users / p.all_users * 100, 2) AS CR_active_control
FROM
(SELECT 
    COUNT(DISTINCT default.peas.st_id) AS all_users 
FROM default.peas
FULL JOIN default.final_project_check USING(st_id)
FULL JOIN default.studs ON default.peas.st_id = default.studs.st_id
WHERE default.peas.st_id IN (SELECT st_id FROM default.peas GROUP BY st_id HAVING SUM(correct) > 10) AND test_grp = 'control' 
) AS p
CROSS JOIN 
(SELECT 
    COUNT(DISTINCT default.final_project_check.st_id) AS payd_users
FROM default.final_project_check
FULL JOIN default.studs ON default.final_project_check.st_id = default.studs.st_id
WHERE test_grp = 'control'
) AS fpc),

sq8 AS (SELECT -- 34.38%  Конверсия из активного юзера в покупателя - test group
        ROUND(fpc.payd_users / p.all_users * 100, 2) AS CR_active_pilot
FROM
(SELECT 
    COUNT(DISTINCT default.peas.st_id) AS all_users 
FROM default.peas
FULL JOIN default.final_project_check USING(st_id)
FULL JOIN default.studs ON default.peas.st_id = default.studs.st_id
WHERE default.peas.st_id IN (SELECT st_id FROM default.peas GROUP BY st_id HAVING SUM(correct) > 10) AND test_grp = 'pilot' 
) AS p
CROSS JOIN 
(SELECT 
    COUNT(DISTINCT default.final_project_check.st_id) AS payd_users
FROM default.final_project_check
FULL JOIN default.studs ON default.final_project_check.st_id = default.studs.st_id
WHERE test_grp = 'pilot'
) AS fpc),

sq9 AS (SELECT -- 16.33% Конверсия активных в математике в покупку курса по математике control group
        ROUND(fpc.payd_users / cau.count_of_math_active_users * 100, 2) AS CRmath_control
FROM
(
SELECT COUNT(count_of_math_students) AS count_of_math_active_users
FROM(
SELECT st_id As count_of_math_students, SUM(correct) As s
FROM default.peas 
FULL JOIN default.studs ON default.peas.st_id = default.studs.st_id
WHERE subject = 'Math' AND test_grp = 'control'
GROUP BY st_id
HAVING s > 2)
) AS cau
CROSS JOIN 
(SELECT 
    COUNT(DISTINCT st_id) AS payd_users
FROM default.final_project_check 
FULL JOIN default.studs ON default.final_project_check .st_id = default.studs.st_id
WHERE subject = 'Math' AND test_grp = 'control'
) AS fpc ),

sq10 AS (SELECT  -- 38.10% Конверсия активных в математике в покупку курса по математике test group
        ROUND(fpc.payd_users / cau.count_of_math_active_users * 100, 2) AS CRmath_pilot
FROM
(
SELECT COUNT(count_of_math_students) AS count_of_math_active_users
FROM(
SELECT st_id As count_of_math_students, SUM(correct) As s
FROM default.peas 
FULL JOIN default.studs ON default.peas.st_id = default.studs.st_id
WHERE subject = 'Math' AND test_grp = 'pilot'
GROUP BY st_id
HAVING s > 2)
) AS cau
CROSS JOIN 
(SELECT 
    COUNT(DISTINCT st_id) AS payd_users
FROM default.final_project_check 
FULL JOIN default.studs ON default.final_project_check .st_id = default.studs.st_id
WHERE subject = 'Math' AND test_grp = 'pilot'
) AS fpc)

SELECT ARPU_control, ARPU_pilot, ARPU_active_control, ARPU_active_pilot, CR_control, CR_pilot, 
CR_active_control, CR_active_pilot, CRmath_control, CRmath_pilot
FROM sq1 
CROSS JOIN sq2
CROSS JOIN sq3
CROSS JOIN sq4
CROSS JOIN sq5
CROSS JOIN sq6
CROSS JOIN sq7
CROSS JOIN sq8
CROSS JOIN sq9
CROSS JOIN sq10
    '''

# отправляем запрос и записываем результат в пандасовский датафрейм
metrics = ph.read_clickhouse(query=q, connection=connection_default)
metrics

,ARPU_control,ARPU_pilot,ARPU_active_control,ARPU_active_pilot,CR_control,CR_pilot,CR_active_control,CR_active_pilot,CRmath_control,CRmath_pilot
0,356506.02,1080902.78,451653.54,1589218.75,9.64,22.6,12.6,34.38,16.33,38.1


In [31]:
# Для более удобного прочтения транспонируем
metrics = metrics.transpose()
metrics = metrics.rename(columns = {metrics.columns[0]: 'values'}) # добавляем имя для столбца
metrics

,values
ARPU_control,356506.02
ARPU_pilot,1080902.78
ARPU_active_control,451653.54
ARPU_active_pilot,1589218.75
CR_control,9.64
CR_pilot,22.60
CR_active_control,12.60
CR_active_pilot,34.38
CRmath_control,16.33
CRmath_pilot,38.10
